# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
tic_data=pd.read_csv('tic-tac-toe.csv')
tic_data.head()


In [ ]:
label_encoder = LabelEncoder()
for column in tic_data.columns:
    tic_data[column] = label_encoder.fit_transform(tic_data[column])
print(tic_data.head())


In [ ]:
data = {
    'TL':    [2, 2, 2, 2, 2],
    'TM':    [2, 2, 2, 2, 2],
    'TR':    [2, 2, 2, 2, 2],
    'ML':    [2, 2, 2, 2, 2],
    'MM':    [1, 1, 1, 1, 1],
    'MR':    [1, 1, 1, 1, 1],
    'BL':    [2, 1, 1, 1, 0],
    'BM':    [1, 2, 1, 0, 1],
    'BR':    [1, 1, 2, 0, 0],
    'class': [1, 1, 1, 1, 1]
}
df = pd.DataFrame(data)
X = df.drop('class', axis=1)  # inputs: TL, TM, …, BR
y = df['class']               # output: class
print("Inputs (X):")
print(X)
print("\nOutput (y):")
print(y)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# X is your DataFrame of inputs 
scaler = MinMaxScaler(feature_range=(0, 1))
X_norm = scaler.fit_transform(X)



## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [55]:
# your code here\
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout  
from tensorflow.keras.optimizers import Adam




In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# Determine number of unique classes for output layer
num_classes = len(np.unique(y_train))

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8,  activation='relu'),
    Dense(num_classes, activation='softmax')  # softmax output for multi-class probabilities
])

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    # ...
    Dense(1, activation='sigmoid'),    # one output neuron with sigmoid
])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',        # binary loss
    metrics=['accuracy']
)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    # ...
    Dense(2, activation='softmax'),    # two outputs for class 0 vs class 1
])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)



c:\Users\Sadiq\OneDrive\Documents\anaconda3\envs\iron_hack_2\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [51]:
# your code here
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('model.h5')

# Randomly select 5 samples from the test set
idx = np.random.choice(len(X_test_scaled), size=5, replace=False)
X_sample = X_test_scaled[idx]
y_true = y_test[idx]

# Get predictions
probs = model.predict(X_sample)
if probs.ndim > 1 and probs.shape[1] > 1:
    y_pred = np.argmax(probs, axis=1)
else:
    y_pred = (probs > 0.5).astype(int).flatten()

# Prepare DataFrame
df = pd.DataFrame({
    'Index': idx,
    'True Label': y_true,
    'Predicted Label': y_pred,
    'Correct': y_pred == y_true
})

ace_tools.display_dataframe_to_user('Test Set Predictions', df)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here